In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import Normalize 
import pandas as pd
import glob
import os
import re
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
import csv
import math
import copy
from scipy.optimize import curve_fit
import scipy.stats as st
import sys
from tqdm import tqdm

In [135]:
a=np.array([0,3,6,5,10,8,1,2,4,7,9])
b=a.argsort()
print(a)
a.sort()
print(a)
print(b)

[ 0  3  6  5 10  8  1  2  4  7  9]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  6  7  1  8  3  2  9  5 10  4]


In [140]:
a=(-1)**(1.5)
print(1/np.sqrt(2*np.pi))

0.3989422804014327
